<a href="https://colab.research.google.com/github/GawainGan/Causal-Inference/blob/main/Causal%20Inference%20and%20Discovery%20in%20Python/Chap_3_Regression%2C_Observations%2C_and_Interventions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# 1. Simple Regression Model

In [3]:
np.random.seed(45)
N_SAMPLES = 5000

In [3]:
alpha = 1.12
beta = 0.93
epsilon = np.random.randn(N_SAMPLES)

In [4]:
X = np.random.randn(N_SAMPLES)
y = alpha + beta * X + 0.5 * epsilon

In [5]:
X[:5]

array([ 0.11530002, -0.43617719, -0.54138887, -1.64773122, -0.32616934])

In [6]:
X = sm.add_constant(X)
X[:5]

array([[ 1.        ,  0.11530002],
       [ 1.        , -0.43617719],
       [ 1.        , -0.54138887],
       [ 1.        , -1.64773122],
       [ 1.        , -0.32616934]])

In [7]:
model = sm.OLS(y, X)
fitted_model = model.fit()
print(fitted_model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.771
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                 1.681e+04
Date:                Fri, 19 Jul 2024   Prob (F-statistic):               0.00
Time:                        03:29:09   Log-Likelihood:                -3615.0
No. Observations:                5000   AIC:                             7234.
Df Residuals:                    4998   BIC:                             7247.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1243      0.007    159.391      0.0

const: 1.1243

x1: 0.9212

p-value: 0 <0.05, thus they are statistically significant

In [13]:
# keep 2 decimal

def fitted_function(x):
  return f'y = {round(fitted_model.params[0], 2)} + {round(fitted_model.params[1], 2)} * x'

print(fitted_function(fitted_model))


y = 1.12 + 0.92 * x


# 2. Capture causal independence between X and Y through regression model

In [4]:
a = np.random.randn(N_SAMPLES)
x = 2 * a + 0.5 * np.random.randn(N_SAMPLES)
y = 2 * a + 0.5 * np.random.randn(N_SAMPLES)
b = 1.5 * x + 0.75 * y

In [6]:
# Define four model variants
variants = [
    [x],
    [x, a],
    [x, b],
    [x, a, b]
    ]

# Fit models iteratively and store the results
results = []
for variant in variants:
  X = sm.add_constant(np.stack(variant).T)
  model = sm.OLS(y, X)
  fitted_model = model.fit()
  results.append(fitted_model)



In [31]:
# Initialize the model dictionary template and model names
model_dict_template = {
    "Model": "",
    "X p-value": "-",
    "X Coefficient": "-",
    "A p-value": "-",
    "A Coefficient": "-",
    "B p-value": "-",
    "B Coefficient": "-"
}

model_names = ["Y ~ X", "Y ~ X + A", "Y ~ X + B", "Y ~ X + A + B"]

# Create an empty list to hold the model dictionaries
model_data = []

# Create a DataFrame with the empty model dictionaries
for model_name in model_names:
    model_dict = model_dict_template.copy()
    model_dict["Model"] = model_name
    model_data.append(model_dict)

# Create DataFrame
combined_df = pd.DataFrame(model_data)
combined_df

,Model,X p-value,X Coefficient,A p-value,A Coefficient,B p-value,B Coefficient
0,Y ~ X,-,-,-,-,-,-
1,Y ~ X + A,-,-,-,-,-,-
2,Y ~ X + B,-,-,-,-,-,-
3,Y ~ X + A + B,-,-,-,-,-,-


In [33]:
# Start with the empty DataFrame from before
combined_df = pd.DataFrame(model_data)

# Manually fill in the data for each row

# Model 1: Y ~ X
combined_df.at[0, "X Coefficient"] = "0.947"
combined_df.at[0, "X p-value"] = "<0.001"

# Model 2: Y ~ X + A
combined_df.at[1, "X Coefficient"] = "0.014"
combined_df.at[1, "X p-value"] = "0.6565"
combined_df.at[1, "A Coefficient"] = "1.967"
combined_df.at[1, "A p-value"] = "<0.001"

# Model 3: Y ~ X + B
combined_df.at[2, "X Coefficient"] = "-2.000"
combined_df.at[2, "X p-value"] = "<0.001"
combined_df.at[2, "B Coefficient"] = "1.333"
combined_df.at[2, "B p-value"] = "<0.001"

# Model 4: Y ~ X + A + B
combined_df.at[3, "X Coefficient"] = "-2.000"
combined_df.at[3, "X p-value"] = "<0.001"
combined_df.at[3, "A Coefficient"] = "0.000"
combined_df.at[3, "A p-value"] = "0.5893"
combined_df.at[3, "B Coefficient"] = "1.333"
combined_df.at[3, "B p-value"] = "<0.001"

combined_df

,Model,X p-value,X Coefficient,A p-value,A Coefficient,B p-value,B Coefficient
0,Y ~ X,<0.001,0.947,-,-,-,-
1,Y ~ X + A,0.6565,0.014,<0.001,1.967,-,-
2,Y ~ X + B,<0.001,-2.000,-,-,<0.001,1.333
3,Y ~ X + A + B,<0.001,-2.000,0.5893,0.000,<0.001,1.333


Only Y ~ X + A shows the causal independence

Large p-valye for X, suggesting the lack of significance

It tells us that all other statistical control schemes led to invalid results, including the model that does not control for any additional variables.

# 3. Regression and structural models

In [35]:
a = np.random.randn(N_SAMPLES)
x = 2 * a + .7 * np.random.randn(N_SAMPLES)
y = 2 * a + 3 * x + .75 * x**2 #这个是我们使用基本公式做出来的 Y~X+A+X^2表达式


X = sm.add_constant(np.stack([x, x**2, a]).T) # 这个是直接用OLS生成的fitted model
model = sm.OLS(y, X)
fitted_model = model.fit()
print(fitted_model.summary(xname=['const', 'x', 'x^2', 'a']))

[-3.08837162 -4.54415465  3.09588805 ...  1.33345169 11.54662926
  0.93710703]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.306e+33
Date:                Fri, 19 Jul 2024   Prob (F-statistic):               0.00
Time:                        23:49:54   Log-Likelihood:             1.5505e+05
No. Observations:                5000   AIC:                        -3.101e+05
Df Residuals:                    4996   BIC:                        -3.101e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

### 回归结果

| Coefficient | Estimated Value | True Value in SCM |
|-------------|------------------|-------------------|
| const       | \(6.162e-15\)    | 0                 |
| x           | \(3.0000\)       | 3                 |
| x^2         | \(0.7500\)       | 0.75              |
| a           | \(2.0000\)       | 2                 |

### 分析
在我们的回归结果中，系数的估计值与我们在生成数据时使用的真值完全一致：

1. **常数项（const）**：理论上应该为 0，但由于计算机浮点数精度问题，估计值接近于 0。
2. **\(x\)**：系数为 3，这与我们在生成 \(y\) 时使用的系数一致。
3. **\(x^2\)**：系数为 0.75，这与我们在生成 \(y\) 时使用的系数一致。
4. **\(a\)**：系数为 2，这与我们在生成 \(y\) 时使用的系数一致。

### 为什么系数一致
这是因为我们在生成 \(y\) 时没有添加任何额外的噪声，\(y\) 完全是 \(x\)、\(x^2\) 和 \(a\) 的确定性函数。这意味着回归分析可以准确地恢复生成数据时的真值系数。

换句话说，我们的模型准确地捕捉了数据的生成过程，因此回归得到的系数与生成数据时使用的系数完全一致。如果我们在生成 \(y\) 时添加噪声，回归系数可能会略有偏离真值，因为噪声会引入误差。

### 总结
- 生成数据时使用的系数是 \(3\)、\(0.75\) 和 \(2\)。
- 回归结果中的系数完全恢复了这些真值。
- 这是因为在生成 \(y\) 时没有添加噪声，因此 \(y\) 是 \(x\)、\(x^2\) 和 \(a\) 的一个确定性函数。